In [18]:
gitURL = 'https://gitlab.aiacademy.tw/junew/federated_aia_test.git'
account = 'at102091:12345678'

In [19]:
import os
if os.getcwd().split('/')[-2:] == ['FL_June', 'server']:
    os.popen('git clone https://{account}@{gitURL}'.format(account=account,
                                                           gitURL=gitURL.split('//')[-1])).read()
    print('clone to ',os.getcwd())
else:
    print(os.getcwd())

clone to  /home/jovyan/git/FL_June/server


In [2]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Input, Dense, Dropout, Activation,
                                     BatchNormalization, Flatten,
                                     Conv2D, MaxPooling2D)
import tensorflow as tf
import numpy as np

from federated_aia_test.utils import compressed_cpickle, decompress_cpickle

### control_key (optional)

In [3]:
contro_key = {}
contro_key['new_model'] = False # default to False


## 移動到federated_aia_test floder
如果不存在，請先執行最上面的git clone

In [4]:
print(os.getcwd())
if os.getcwd().split('/')[-2:] == ['server', 'federated_aia_test']:
    pass
else:
    os.chdir('../../FL_June/server/federated_aia_test')
    os.getcwd()

/home/jovyan/git/FL_June/server


## 建立新的初始化global model 
> 只有當模型不存在、或者你更新了架構、打算重新訓練的時候

In [5]:
def simplecnn():
    # 選擇 Keras 的 API 寫法
    inputs = Input(shape=(28,28,1))
#     inputs = inputs
    # 第一層
    # 建立卷積層，設定32個3*3的filters
    # 設定ReLU為激活函數。
    x = Conv2D(32, (3, 3), activation='relu')(inputs)

    # 第二層 - 卷積層 + 池化層
    x = Conv2D(32, (3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # 第三層 - 卷積層
    x = Conv2D(64, (3, 3), activation='relu')(x)

    # 第四層 - 卷積層 + 池化層
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # 建立分類模型 (MLP) : 平坦層 + 輸出層 (10)
    x = Flatten()(x)
    outputs = Dense(10, activation='softmax')(x)

    cnn_model = Model(inputs=inputs, outputs=outputs)
    return cnn_model

In [6]:
contro_key['new_model'] = True
contro_key

{'new_model': True}

In [17]:


today = date.today()
print(today)

2021-11-27


In [26]:
from datetime import date

lis = os.listdir('../saved_model')
lis = [i for i in lis if i.__contains__('global_model')]



'global_model_3_2021-11-27'

In [31]:
if contro_key['new_model'] == True:
    import os
    import shutil
    from datetime import date
    
    # 保留舊模型到server本機 (saved_model被登錄在.gitignore)
    os.makedirs('../saved_model', exist_ok=True)
    if os.listdir().__contains__('global_model.pbz2'):
        today = date.today()
        lis = os.listdir('../saved_model')
        lis = [i for i in lis if i.__contains__('global_model')]
        
        new_model_name = 'global_model_{ver}_{today}.pbz2'.format(today=today, ver=len(lis))
        shutil.move('./global_model.pbz2', '../saved_model/'+new_model_name)
        print('Move global model to FL_June/server/saved_model')
    else:
        pass
    model = simplecnn()

    # weight、架構 (json) to pickle

    model_attri = {'weights':model.get_weights(), 'json':model.to_json()}
    
    print('create new model')
    compressed_cpickle('./global_model', model_attri)

    print('update gitrepo global_model.pbz2')
    run_cmd = lambda cmd_lis:[os.popen(i).read() for i in cmd_lis.split('\n')]
    cmd_lis = '''git add .
    git commit -m'global model complete aggregate and update to g'
    git push https://{account}@{gitURL}
    '''.format(account=account, gitURL=gitURL.split('//')[-1])
    
    
    run_cmd(cmd_lis)


Move global model to FL_June/server/saved_model
update gitrepo global_model.pbz2


## 下載各個branch中的模型壓縮檔

In [46]:
r = os.popen('git pull').read()
print(r)

os.popen('git remote update origin --prune')
lis = os.popen('git branch -r').read().split('\n')[:-1]


all_client_branch = [i for i in  lis if not i.__contains__('master')]

print(all_client_branch)

# 判斷各個分支是否有更新
if len(all_client_branch) <= 0:
    raise ValueError('June: No clients appear')

['  origin/KJugrbO2x4I0JwkYIaLNFQ', '  origin/T_gHdTLAsBgPd-mt3Vx-3w']


In [47]:
run_cmd = lambda cmd_lis:[os.popen(i).read() for i in cmd_lis.split('\n')]

for i in all_client_branch:
    origin_branch_name = i.lstrip()
    filename = origin_branch_name.split('/')[-1]+'.pbz2'
    
    print(origin_branch_name, filename)
    
    
    cmd_lis = '''git checkout remotes/{branch_name} {model_attri_pbz2}
    '''.format(branch_name = origin_branch_name, model_attri_pbz2=filename)
    result = os.popen(cmd_lis).read()
    print(result)

origin/KJugrbO2x4I0JwkYIaLNFQ KJugrbO2x4I0JwkYIaLNFQ.pbz2

origin/T_gHdTLAsBgPd-mt3Vx-3w T_gHdTLAsBgPd-mt3Vx-3w.pbz2



In [48]:
!git branch -a

* master
  remotes/origin/HEAD -> origin/master
  remotes/origin/KJugrbO2x4I0JwkYIaLNFQ
  remotes/origin/T_gHdTLAsBgPd-mt3Vx-3w
  remotes/origin/master


## 聚合並更新global model

In [49]:
model_attri = decompress_cpickle('./global_model.pbz2')
global_model = tf.keras.models.model_from_json(model_attri['json'])

lis = [i for i in os.listdir() if i.__contains__('pbz2')]
lis.remove('global_model.pbz2')
print(lis)

['KJugrbO2x4I0JwkYIaLNFQ.pbz2', 'T_gHdTLAsBgPd-mt3Vx-3w.pbz2']


In [50]:
weights = []
for i in lis:
    model_attri = decompress_cpickle(i)
    weights.append(model_attri['weights'])
print(np.shape(weights))

(2, 10)


In [51]:
new_weights = list()
if len(weights) == 0:
    print('no new client to aggregate')
    pass
elif len(weights) == 1:
    print('only single participant')
    new_weights = weights[0]
else:
    for i in zip(*weights):
        new_weights.append(tf.reduce_sum(i, axis=0))
        
global_model.set_weights(model_attri['weights'])

In [52]:
model_attri = {'weights':global_model.get_weights(), 'json':global_model.to_json()}

compressed_cpickle('./global_model', model_attri)

In [53]:
import os

run_cmd = lambda cmd_lis:[os.popen(i).read() for i in cmd_lis.split('\n')]

cmd_lis = '''mv ./global_model.pbz2 ../
rm *.pbz2
mv ../global_model.pbz2 ./
git add .
git commit -m'global model complete aggregate and update to Gmodel'
git push https://{account}@{gitURL}
'''.format(account=account, gitURL=gitURL.split('//')[-1])


run_cmd(cmd_lis)
branch = os.popen('git branch -a').read()
print(branch)

* master
  remotes/origin/HEAD -> origin/master
  remotes/origin/KJugrbO2x4I0JwkYIaLNFQ
  remotes/origin/T_gHdTLAsBgPd-mt3Vx-3w
  remotes/origin/master

